In [62]:
#adapted from @bact at https://colab.research.google.com/drive/1hdtmwTXHLrqNmDhDqHnTQGpDVy1aJc4t
import json
import pandas as pd
import numpy as np
import re
import pycrfsuite
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
with open('data/talks.transcript.th-en.24-10-2019_11:12.json', 'r') as f:
    data = json.load(f)

In [63]:
talk_idx = []
idx = []
th_phrases = []
en_phrases = []

for i in range(len(data)):
    nb_phrases = min(len(data[i]['en']),len(data[i]['th']))
    for j in range(nb_phrases):
        talk_idx.append(i)
        th_phrases.append(data[i]['th'][j])
        en_phrases.append(data[i]['en'][j])
        if data[i]['en'][j][-1]=='.':
            idx.append(1)
        else:
            idx.append(0)

In [64]:
phrase_df = pd.DataFrame({'talk_idx':talk_idx,'en_phrase':en_phrases ,'th_phrase':th_phrases, 'idx':idx})
phrase_df.idx.sum()

136463

In [65]:
all_sentences = []
for i in range(phrase_df.talk_idx.max()):
    df = phrase_df[phrase_df.talk_idx==i]
    sentences = []
    for j,row in df.iterrows():
        sentences.append(row.th_phrase)
        if row.idx==1:
            sentences.append(' |')
    joined_sentences = ''.join(sentences)
    #remove parantheses like (audience claps)
    joined_sentences = re.sub(r'\([^)]*\)', '', joined_sentences)
    #skip if talk is nothing but parantheses
    if joined_sentences=='': continue
    #remove | at the last sentence if present
    if joined_sentences[-1]=='|': joined_sentences = joined_sentences[:-1]
    all_sentences.append(joined_sentences)

In [7]:
len(all_sentences)

1544

In [66]:
all_sentences[0]

'บนหลังอาชาแห้งกร่องดอนกิโฆเต้ พระเอกของเราบุกตะลุยสู้กับกองทัพยักษ์ |ในสายตาของเขา มันเป็นหน้าที่ของเขาที่จะปราบอสูรร้ายเหล่านี้ในนามแห่งหญิงอันเป็นที่รักของเขา ดุลสิเนอา |ทว่า การกระทำอันหาญกล้านี้ก็สูญเปล่า |เมื่อซานโซ่ ปันซ่า ผู้รับใช้ของเขาอธิบายครั้งแล้วครั้งเล่า ว่าสิ่งเหล่านี้จะเป็นยักษ์ก็หาไม่พวกมันเป็นเพียงกังหันลมเท่านั้น |ดอนกิโฆเต้ หาได้เสียความแน่วแน่แทงทวนของเขาเข้าไปยังใบพัดอย่างจัง |ด้วยพลังใจที่ไม่เคยถดถอยอัศวินผู้นั้นยืนขึ้นอย่างภาคภูมิและยิ่งเชื่อมั่นในปฏิบัติการของเขามากขึ้น |ลำดับเหตุการณ์นี้ครอบคลุมเรื่องราวส่วนใหญ่ของดอนกิโฆเต้ ที่เป็นที่รักมหากาพย์ ไร้ตรรกะ และมีชีวิตชีวาของ อลองโซ กีฆานาผู้กลายเป็น ดอนกิโฆเต้ แห่งลามันช่าผู้ซุ่มซ่ามแต่กล้าหาญหรือที่รู้จักกันในนาม ขุนนางต่ำศักดิ์นักฝัน |แต่เดิมวรรณกรรมนี้มีสองเล่มบรรยายเรื่องราวของดอนกิโฆเต้ในขณะที่เขาเดินทางผ่านตอนกลาง และตอนเหนือของสเปนเพื่อต่อสู้กับบรรดาปิศาจร้าย |แม้จินตนาการในเรื่อง ดอนกิโฆเต้ จะสูงล้ำเหนือเมฆผู้ประพันธ์ มิเกล เด เซร์บันเตสก็ไม่เคยนึกฝันว่าหนังสือของเขาจะกลายเป็นนิยายที่ขายดีที่สุดตลอดกาล 

In [67]:
all_tuples = []
for i in range(len(all_sentences)):
    tuples = []
    for s in all_sentences[i].split('|'):
        s_lst = word_tokenize(s)
        for j in range(len(s_lst)):
            lab = 'B' if j==0 else 'I'
            tuples.append((s_lst[j],lab))
    all_tuples.append(tuples)

In [68]:
len(all_tuples)

1544

In [69]:
# Extract features of each wordacter from text, in CRFSuite format
def extract_features(doc):
    doc_pos = [p for (w,p) in pos_tag(doc)]
    doc_features = []
    for i, word in enumerate(doc):
        word_features = [
            'bias',
            'word=' + word,
            'pos=' + doc_pos[i],
        ]
        if i > 0:
            feats = [
                'word[-1]=' + doc[i-1],
                'pos[-1]=' + doc_pos[i-1],
            ]
            word_features.extend(feats)

        if i > 1:
            feats = [
                'word[-2]=' + doc[i-2],
                'pos[-2]=' + doc_pos[i-2],
            ]
            word_features.extend(feats)

        if i < len(doc)-1:
            feats = [
                'word[+1]=' + doc[i+1],
                'pos[+1]=' + doc_pos[i+1],
            ]
            word_features.extend(feats)
            
        if i < len(doc)-2:
            feats = [
                'word[+2]=' + doc[i+2],  
                'pos[+2]=' + doc_pos[i+2],
            ]
            word_features.extend(feats)
        doc_features.append(word_features)
    return doc_features

In [70]:
#target
y = [[l for (w,l) in t] for t in all_tuples]
#features
x_pre = [[w for (w,l) in t] for t in all_tuples]
x = [extract_features(x_) for x_ in x_pre]

In [71]:
len(x),len(y)

(1544, 1544)

In [72]:
# Split train and test set at 80/20 proportion
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1412)

In [73]:
# Train model
trainer = pycrfsuite.Trainer(verbose=True)
#trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(x_train, y_train):
  trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 0.1,
    'c2': 0.01,
    'max_iterations': 200,
    'feature.possible_transitions': True,
})

trainer.train('sentenceseg-crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 187577
Seconds required: 7.417

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 458436.018006
Feature norm: 1.000000
Error norm: 231630.780318
Active features: 184987
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.676

***** Iteration #2 *****
Loss: 443295.586541
Feature norm: 1.054247
Error norm: 204125.242882
Active features: 182163
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.797

***** Iteration #3 *****
Loss: 366649.934647
Feature norm: 1.389798
Error norm: 199300.213013
Active features: 130312
Line search trials: 1
Line search step: 1.000000
Seconds re

***** Iteration #39 *****
Loss: 142065.029644
Feature norm: 37.695884
Error norm: 7394.078349
Active features: 89155
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.897

***** Iteration #40 *****
Loss: 140150.890203
Feature norm: 41.993974
Error norm: 10367.979278
Active features: 88985
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.768

***** Iteration #41 *****
Loss: 138369.443375
Feature norm: 46.758963
Error norm: 8997.034539
Active features: 88892
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.834

***** Iteration #42 *****
Loss: 136576.784972
Feature norm: 52.997569
Error norm: 7352.372174
Active features: 88630
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.752

***** Iteration #43 *****
Loss: 134707.365693
Feature norm: 61.226410
Error norm: 11251.364606
Active features: 88373
Line search trials: 1
Line search step: 1

***** Iteration #78 *****
Loss: 113106.579458
Feature norm: 316.586458
Error norm: 6428.812295
Active features: 78156
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.967

***** Iteration #79 *****
Loss: 113044.074246
Feature norm: 316.897801
Error norm: 6083.911621
Active features: 78030
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.828

***** Iteration #80 *****
Loss: 112982.678384
Feature norm: 317.304252
Error norm: 6548.279134
Active features: 78003
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.920

***** Iteration #81 *****
Loss: 112925.397617
Feature norm: 317.632152
Error norm: 6149.454491
Active features: 77977
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.867

***** Iteration #82 *****
Loss: 112872.027681
Feature norm: 318.023601
Error norm: 7128.425400
Active features: 77920
Line search trials: 1
Line search step

***** Iteration #119 *****
Loss: 111528.635912
Feature norm: 324.908874
Error norm: 3251.507181
Active features: 77345
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.861

***** Iteration #120 *****
Loss: 111504.851028
Feature norm: 324.989966
Error norm: 3032.252843
Active features: 77331
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.885

***** Iteration #121 *****
Loss: 111483.469016
Feature norm: 325.049993
Error norm: 3494.339634
Active features: 77324
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.511

***** Iteration #122 *****
Loss: 111458.840532
Feature norm: 325.134045
Error norm: 2845.124190
Active features: 77321
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.825

***** Iteration #123 *****
Loss: 111438.468723
Feature norm: 325.189838
Error norm: 3316.760751
Active features: 77317
Line search trials: 2
Line search

***** Iteration #159 *****
Loss: 110715.112880
Feature norm: 327.451211
Error norm: 2791.528420
Active features: 76812
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.514

***** Iteration #160 *****
Loss: 110698.410085
Feature norm: 327.470851
Error norm: 2256.263642
Active features: 76790
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.484

***** Iteration #161 *****
Loss: 110684.974810
Feature norm: 327.442410
Error norm: 2654.206914
Active features: 76766
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.641

***** Iteration #162 *****
Loss: 110669.111254
Feature norm: 327.418087
Error norm: 2283.299021
Active features: 76736
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.570

***** Iteration #163 *****
Loss: 110655.446808
Feature norm: 327.334474
Error norm: 2588.795552
Active features: 76711
Line search trials: 2
Line search

In [76]:
# Predict (using test set)
tagger = pycrfsuite.Tagger()
tagger.open('sentenceseg-crf.model')
y_pred = [tagger.tag(xseq) for xseq in x_test]

In [77]:
# Evaluate
labels = {'B': 0, "I": 1} # classification_report() needs values in 0s and 1s
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["B", "I"]))

              precision    recall  f1-score   support

           B       0.68      0.67      0.67     27763
           I       0.99      0.99      0.99    649564

    accuracy                           0.97    677327
   macro avg       0.83      0.83      0.83    677327
weighted avg       0.97      0.97      0.97    677327



In [107]:
def sentence_tokenize(s):
    toks = word_tokenize(s)
    feat = extract_features(toks)
    labs = tagger.tag(feat)
    sentences = []
    sentence = ''
    for i, w in enumerate(toks):
        if labs[i] == 'B':
            if sentence:
                sentences.append(sentence)
            sentence = ''
        sentence = sentence + w
    if sentence:
        sentences.append(sentence)
    return sentences

In [108]:
s = 'เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์ เธอสามารถตัดสินใจด้วยตัวเอง แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์ ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก 30 ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์ แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ จะพร้อมเสียก่อน'

In [109]:
sentence_tokenize(s)

['เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์<space>',
 'เธอสามารถตัดสินใจด้วยตัวเอง<space>',
 'แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม<space>',
 'ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย<space>จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์<space>',
 'ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก<space>30<space>ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์<space>',
 'แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น<space>',
 'ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ<space>จะพร้อมเสียก่อน']